Para poder entrenar nuestros modelos de machine learning debemos crear un archivo de ingeniería de características.  



Text Cleaning and Preparation: cleaning of special characters, downcasing, punctuation signs. possessive pronouns and stop words removal and lemmatization.
Label coding: creation of a dictionary to map each category to a code.
Train-test split: to test the models on unseen data.
Text representation: use of TF-IDF scores to represent text.

la ingeniería de características es el proceso de usar el conocimiento de dominio de los datos para crear características que hacen que los algoritmos de aprendizaje automático funcionen. Es como un arte, ya que requiere conocimiento del dominio y puede ser difícil crear características, pero puede ser fructífero para el algoritmo ML para predecir los resultados, ya que pueden estar relacionados con la predicción.

fuente: https://towardsdatascience.com/natural-language-processing-nlp-for-machine-learning-d44498845d5b

# Importación de las librerías

In [1]:
import pandas as pd
import seaborn as sns
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from emoji import UNICODE_EMOJI
from nltk import bigrams 
import pickle

# Lectura de la muestra

In [2]:
df = pd.read_csv("../00_Dataset/clean_sentiment_labels_completo.csv", sep=';')
df.head()
df.is_copy = False

C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\generic.py:4384: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\generic.py:4385: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


# Limpieza de los datos
Una limpieza de los datos ha sido realizada tanto en el EDA como en la extracción. Sin embargo, aún hay algunos detalles que debemos depurar para poder hacer un buen análisis sobre nuestros datos.  

Es  habitual la supresión de signos de puntuación pero en este caso nos ayudan a identificar la intensidad de los comentarios.

**Supresión de las columnas no necesarias***

In [3]:
df_eng=df[['texto', 'series_token','num_may_esc','num_insultos','score_emoji_tox','emphasize','toxico']]
df.head()

,texto,toxico,num_admir,num_may,num_may_esc,emojis,num_emoji,num_insultos,dir_23ps,score_emoji_tox,emphasize,series_token2,series_token,filtered_words
0,"la etiqueta, cabron",1,0,0,0.0,NaN,0,1,0,0.0,0.0,"la etiqueta, cabron","['la', 'etiqueta', ',', 'cabron']","['etiqueta,', 'cabron']"
1,hasta que un cabron trata a la lacra como se d...,1,0,0,0.0,NaN,0,1,0,0.0,0.0,hasta que un cabron trata a la lacra como se d...,"['hasta', 'que', 'un', 'cabron', 'trata', 'a',...","['cabron', 'trata', 'lacra', 'debe', 'indignan']"
2,la edad no la pones eh cabron,1,0,0,0.0,NaN,0,1,1,0.0,1.0,la edad no la pones eh cabron,"['la', 'edad', 'no', 'la', 'pones', 'eh', 'cab...","['edad', 'no', 'pones', 'eh', 'cabron']"
3,y de un cabron solo puedes obtener cabronadas,1,0,0,0.0,NaN,0,1,1,0.0,1.0,y de un cabron solo puedes obtener cabronadas,"['y', 'de', 'un', 'cabron', 'solo', 'puedes', ...","['cabron', 'solo', 'puedes', 'obtener', 'cabro..."
4,eres mas guiri que los irlandeses de aqui cabr...,1,0,1,0.1,NaN,0,1,1,0.0,1.1,eres mas guiri que los irlandeses de aqui cabr...,"['eres', 'mas', 'guiri', 'que', 'los', 'irland...","['mas', 'guiri', 'irlandeses', 'aqui', 'cabron..."


**Redondeo del valor emphasize**

Donde 70.55 es 71 y 70.4 es 70

In [4]:
round(float(1.020408))

1

In [5]:
df_eng.loc[:,'emphasize']=df_eng.loc[:,'emphasize'].map(lambda value: round(float(value),1) )

C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
df_eng[df_eng['emphasize']!=0.0].head()

,texto,series_token,num_may_esc,num_insultos,score_emoji_tox,emphasize,toxico
2,la edad no la pones eh cabron,"['la', 'edad', 'no', 'la', 'pones', 'eh', 'cab...",0.0,1,0.0,1.0,1
3,y de un cabron solo puedes obtener cabronadas,"['y', 'de', 'un', 'cabron', 'solo', 'puedes', ...",0.0,1,0.0,1.0,1
4,eres mas guiri que los irlandeses de aqui cabr...,"['eres', 'mas', 'guiri', 'que', 'los', 'irland...",0.1,1,0.0,1.1,1
7,que es eso cabron,"['que', 'es', 'eso', 'cabron']",0.0,1,0.0,1.0,1
8,alba “este hijo de puta de platini lo hicieron...,"['alba', '“', 'este', 'hijo', 'de', 'puta', 'd...",0.0,1,0.0,1.0,1


**Supresión de abreviaturas en el texto y conservión a minúsculas**

In [7]:
#Creamos un diccionario desde el csv que hemos creado
path_ab='../00_Dataset/abreviaturas.csv'
abrev = pd.read_csv(path_ab, sep=';')
diccionario_abr=abrev.set_index('abreviatura').T.to_dict('dict')

In [8]:
#Creación de una función en el que palabras como xa se convierten en para
def delete_abrev(string):
    words=string.split()
    for i in range(len(words)):
        if words[i] in diccionario_abr.keys():
            words[i] = diccionario_abr[words[i]]['significado'].lower()
    return " ".join(words)



#Comprobación del funcionamiento
aux=df_eng.iloc[48]['texto']
print(aux)
delete_abrev(aux)

parguela vete ya que estabamos contentos pensando que te habias muerto


'parguela vete ya que estabamos contentos pensando que te habias muerto'

In [9]:
#Sustitución de las abreviaciones en el dataset original
df_eng.loc[:,'texto']=df_eng.loc[:,'texto'].map(lambda x: delete_abrev(x))
df_eng.is_copy = False
df_eng.head()

C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\generic.py:4384: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\generic.py:4385: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


,texto,series_token,num_may_esc,num_insultos,score_emoji_tox,emphasize,toxico
0,"la etiqueta, cabron","['la', 'etiqueta', ',', 'cabron']",0.0,1,0.0,0.0,1
1,hasta que un cabron trata a la lacra como se d...,"['hasta', 'que', 'un', 'cabron', 'trata', 'a',...",0.0,1,0.0,0.0,1
2,la edad no la pones eh cabron,"['la', 'edad', 'no', 'la', 'pones', 'eh', 'cab...",0.0,1,0.0,1.0,1
3,y de un cabron solo puedes obtener cabronadas,"['y', 'de', 'un', 'cabron', 'solo', 'puedes', ...",0.0,1,0.0,1.0,1
4,eres mas guiri que los irlandeses de aqui cabr...,"['eres', 'mas', 'guiri', 'que', 'los', 'irland...",0.1,1,0.0,1.1,1


**Tokenización quitando las stopwords y los signos de puntuación**

No siempre se han de quitar todas las stopwords ya que algunas como no si tienen peso en el algoritmo. En este caso eliminamos no del conjunto de palabras vacías y añadimos los signos de puntuación por tenerlos ya en cuenta en la etiqueta emphasize. 

Porqué eliminar las stopwords siempre no es una buena idea. Fuente: https://medium.com/@wilamelima/why-is-removing-stop-words-not-always-a-good-idea-c8d35bd77214


In [10]:
stopWords = stopwords.words('spanish')
stopWords.remove('no')

df_eng.loc[:,'filtered_words']=df_eng.loc[:,'texto'].map(lambda texto: texto.replace('.',''))
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda texto: texto.replace('?',''))
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda texto: texto.replace('!',''))
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda texto: texto.replace('¿',''))
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda texto: texto.replace(',',''))
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda texto: texto.replace('“',''))
#Supresión de las stopword en castellano
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].apply(lambda frase: [word.lower() for word in frase.split(' ') if word.lower() not in stopWords])

df_eng.loc[38,'filtered_words']


['asco',
 'periodismo',
 'cadena',
 'perroflauta',
 'miserables',
 'personajes',
 '😠',
 '😡',
 '😠']

**Lemmanization**

Lemmatización es el proceso de convertir una palabra a su forma básica. La diferencia entre stemming y lemmatization es que la lematización considera el contexto y convierte la palabra a su forma básica significativa, mientras que stemming simplemente elimina los últimos caracteres, lo que a menudo conduce a significados incorrectos y errores de ortografía.

fuente: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [11]:
#Prueba de funcionamiento
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("spanish")
stemmer.stem("estudiando")

'estudi'

In [12]:
import spacy
nlp = spacy.load('es_core_news_sm')
text1 = "Estoy estudiando como funciona lemma en español y como no funciona.Pero que dices zampabollos "
res=[]
for token in nlp(text1):
    res.append(token.lemma_)
print(res)

['Estoy', 'estudiar', 'comer', 'funcionar', 'lemma', 'en', 'español', 'y', 'comer', 'no', 'funcionar', '.', 'Pero', 'que', 'decir', 'zampabollos']


In [13]:
#Definición de la función que extrae el origen de la palabra
def extraccion_lemma(text): 
    str1 = ' '.join(str(e) for e in text)
    res=[]
    for word in nlp(str1):
        res.append(word.lemma_)
    return res



Podemos ver que la raiz no es tomada correctamente y no proporciona valor. Pero el lemma funciona suficientemente bien en castellano.  

Algunos bugs como los presentados en https://github.com/explosion/spaCy/issues/2710 siguen sin ser solucionados. Palabras como 'estoy' no son traducidas a estar, 'como' es traducido por comer,.... 

In [14]:
df_eng.loc[:,'filtered_words_lemm']=df_eng.loc[:,'filtered_words'].map(lambda text: extraccion_lemma(text) )
df_eng.head()

,texto,series_token,num_may_esc,num_insultos,score_emoji_tox,emphasize,toxico,filtered_words,filtered_words_lemm
0,"la etiqueta, cabron","['la', 'etiqueta', ',', 'cabron']",0.0,1,0.0,0.0,1,"[etiqueta, cabron]","[etiquetar, cabron]"
1,hasta que un cabron trata a la lacra como se d...,"['hasta', 'que', 'un', 'cabron', 'trata', 'a',...",0.0,1,0.0,0.0,1,"[cabron, trata, lacra, debe, indignan]","[cabron, tratar, lacrar, deber, indignar]"
2,la edad no la pones eh cabron,"['la', 'edad', 'no', 'la', 'pones', 'eh', 'cab...",0.0,1,0.0,1.0,1,"[edad, no, pones, eh, cabron]","[edad, no, poner, eh, cabron]"
3,y de un cabron solo puedes obtener cabronadas,"['y', 'de', 'un', 'cabron', 'solo', 'puedes', ...",0.0,1,0.0,1.0,1,"[cabron, solo, puedes, obtener, cabronadas]","[cabron, solo, poder, obtener, cabronada]"
4,eres mas guiri que los irlandeses de aqui cabr...,"['eres', 'mas', 'guiri', 'que', 'los', 'irland...",0.1,1,0.0,1.1,1,"[mas, guiri, irlandeses, aqui, cabron, xd]","[mas, guiri, irlandés, aqui, cabron, xd]"


# Dividiendo el dataset en train, dev and test sets 
Pasos a tener en cuenta: 
- dev y test deben proceder de la misma distribución y se debe tomar de forma aleatoria del total del corpus. 
- dev y test deben tener el mismo tamaño. En nuestro caso dividiremos en train 70%, dev 15% y test 15%.
- Se quiere tener unos datos no sesgados a la hora de realizar el algoritmo, para ello se cambia el orden de las filas pero inicializando una semilla que fije los resultados dependientes de una variable aleatoria. 


fuente: https://www.coursera.org/learn/machine-learning-projects

In [15]:
df_eng.loc[38,'filtered_words']

['asco',
 'periodismo',
 'cadena',
 'perroflauta',
 'miserables',
 'personajes',
 '😠',
 '😡',
 '😠']

In [16]:
dfmod=df_eng.reindex(np.random.RandomState(seed=42).permutation(df_eng.index))


In [17]:
from sklearn.model_selection import train_test_split
columnas=['filtered_words','filtered_words_lemm','num_insultos', 'score_emoji_tox','emphasize','toxico']
dfimp=dfmod.loc[:,columnas]
X=dfmod.loc[:,columnas]
y=dfmod.loc[:,'toxico']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
#everytime you run it without specifying random_state, you will get a different result
print( X_train.shape, y_train.shape) 
print( X_test.shape, y_test.shape)
X_train.iloc[240]

(564, 6) (564,)
(141, 6) (141,)


filtered_words          [pongo, castigo, cabezabuque]
filtered_words_lemm    [poner, castigar, cabezabuque]
num_insultos                                        1
score_emoji_tox                                     0
emphasize                                           1
toxico                                              1
Name: 372, dtype: object

# Guardado de los datos

Pickle es utilizado cuando se tiene un conjunto de datos grande y se carga ese conjunto de datos masivos en la memoria cada vez que ejecuta el programa.Almacenarlo y recuperarlo será mucho más rápido, del orden de 50 - 100x, a veces mucho más dependiendo del tamaño.

fuente: https://pythonprogramming.net/python-pickle-module-save-objects-serialization/


In [18]:
dfimp.loc[38,'filtered_words']

['asco',
 'periodismo',
 'cadena',
 'perroflauta',
 'miserables',
 'personajes',
 '😠',
 '😡',
 '😠']

In [19]:
#Almacenamiento del df del modelo
pickle_out = open("dfmodelo.pickle","wb")
pickle.dump(dfimp, pickle_out)
pickle_out.close()

#Almacenamiento del x_train
pickle_out = open("X_train.pickle","wb")
pickle.dump(X_train, pickle_out)
pickle_out.close()

#Almacenamiento del x_test
pickle_out = open("X_test.pickle","wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

#Almacenamiento del y_train
pickle_out = open("y_train.pickle","wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

#Almacenamiento del y_test
pickle_out = open("y_test.pickle","wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()